# Enabling Complex Reasoning and Action with ReAct, LLMs, and LangChain

**NOTEBOOK KERNEL REQUIREMENTS** 

Please ensure you are using the following settings for your notebook environment. 

![notebook-environment](notebook-environment.png)

---

In this notebook you will learn how to use multiple different techniques and models to build a ReAct based framework. ReAct is an approach to problem solving with large language models based on 2 main premises: Reasoning and Action. With ReAct, you combine reasoning, through chain-of-thought, with the ability to perform actions through a set of tools. This enables the model to (Re)ason through the input request to determine what steps need to be performed, and uses the available tools to perform (ACT)ions as part of a step-by-step resolution.

More details on ReAct can be found in this research paper: [ReAct: Synergizing Reasoning and Acting in Language Models](https://arxiv.org/abs/2210.03629) and the [Google AI Blog](https://blog.research.google/2022/11/react-synergizing-reasoning-and-acting.html)

![image.png](ReAct-Diagram.png)

Image taken from Google AI Blog

To demonstrate the potential of ReAct this notebook will focus on a use case involving and Insurance Bot.  For demonstration purposes, this Bot is designed to handle insurance policy requests and is provided a set of tools, including a SQLLite database and an insurance processing API, to accept requests on input, reason through the steps needed to carry out the request, and carry out the actions required.  

## Dependencies

As part of this workshop you will run a local, quantized version of Meta's LLaMa-2-13B. In order do do this you will need llama-cpp-python, which requires a local compiler. This will install `g++` and set it as the default so that you can install llama-cpp-python.

In [2]:
!apt-get update
!apt-get -y install g++
%env CXX=g++

Hit:1 http://security.debian.org/debian-security bullseye-security InRelease
Hit:2 http://deb.debian.org/debian bullseye InRelease
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Fetched 44.1 kB in 0s (181 kB/s)    
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:10.2.1-1).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
env: CXX=g++


In [3]:
%pip install ipywidgets==8.1.1
%pip install transformers[torch]==4.34.1
%pip install sentence_transformers==2.2.2
%pip install langchain==0.0.324
%pip install llama-cpp-python==0.2.11
%pip install faiss-cpu==1.7.4
%pip install langchain-experimental==0.0.36
%pip install sqlalchemy==1.4.39


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgr

# Workshop Start

Setting `langchain.debug = True` allows you to see the deep dive internals of what is happening under the hood of your chains and agents.

In [4]:
import langchain

langchain.debug = True

## Setup sample data using SQLite

As part of this workshop, you will see how to work with a variety of different tool types to retrieve and act on data.

Here you will create an in-memory SQLite database to store some sample data for the examples.

In [5]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

Build out a table of insurance policies. This would typically be normalized, but for simplicity's sake of this example, the policies and users are all in 1 table.

In [6]:
from sqlalchemy import Column, Integer, String, Table, Date

policies = Table(
    "policies",
    metadata_obj,
    Column("policy_id", Integer, primary_key=True),
    Column("first_name", String(50), nullable=False),
    Column("last_name", String(50), nullable=False),
    Column("phone", String(15), nullable=False),
    Column("policy_type", String(25), nullable=False),
    Column("policy_date", Date, nullable=False),
    Column("policy_value", Integer, nullable=False),
)

In [7]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

Generate some some dummy data and insert it into the database.

In [8]:
from datetime import datetime

#policy data: policy_id, first_name, last_name, phone, policy_type, policy_date, policy_value

policy_data = [
    [48918, 'Ernest', 'Mcneil', '349-711-8757', 'home', datetime(2023, 1, 1), 250000],
    [66958, 'Brian', 'Patel', '368-889-1742', 'auto', datetime(2023, 1, 2), 32000],
    [21947, 'Bertram', 'Mcgee', '798-641-5925', 'home', datetime(2023, 1, 3), 550000],
    [17108, 'Margarito', 'Rollins', '348-321-5711', 'auto', datetime(2023, 1, 4), 75000],
    [98362, 'Miriam', 'Sutton', '361-863-4332', 'auto', datetime(2023, 1, 8), 21000],
    [17565, 'Charmaine', 'Hopkins', '206-566-6359', 'home', datetime(2023, 1, 2), 135000],
    [10157, 'Jewel', 'Ingram', '598-338-6133', 'home', datetime(2023, 1, 6), 750000],
    [33372, 'Kaye', 'Underwood', '555-720-3848', 'home', datetime(2023, 1, 1), 235000],
    [97143, 'Josiah', 'Vazquez', '211-391-1757', 'auto', datetime(2023, 1, 5), 17250],
    [54621, 'Charles', 'Wise', '502-236-0425', 'home', datetime(2023, 1, 4), 1592000],
]

In [9]:
from sqlalchemy import insert

def insert_policy_data(policy_data_arr):
    stmt = insert(policies).values(
    policy_id=policy_data_arr[0],
    first_name=policy_data_arr[1],
    last_name=policy_data_arr[2],
    phone=policy_data_arr[3],
    policy_type=policy_data_arr[4],
    policy_date=policy_data_arr[5],
    policy_value=policy_data_arr[6]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [10]:
for policy in policy_data:
    print(policy)
    insert_policy_data(policy)

[48918, 'Ernest', 'Mcneil', '349-711-8757', 'home', datetime.datetime(2023, 1, 1, 0, 0), 250000]
[66958, 'Brian', 'Patel', '368-889-1742', 'auto', datetime.datetime(2023, 1, 2, 0, 0), 32000]
[21947, 'Bertram', 'Mcgee', '798-641-5925', 'home', datetime.datetime(2023, 1, 3, 0, 0), 550000]
[17108, 'Margarito', 'Rollins', '348-321-5711', 'auto', datetime.datetime(2023, 1, 4, 0, 0), 75000]
[98362, 'Miriam', 'Sutton', '361-863-4332', 'auto', datetime.datetime(2023, 1, 8, 0, 0), 21000]
[17565, 'Charmaine', 'Hopkins', '206-566-6359', 'home', datetime.datetime(2023, 1, 2, 0, 0), 135000]
[10157, 'Jewel', 'Ingram', '598-338-6133', 'home', datetime.datetime(2023, 1, 6, 0, 0), 750000]
[33372, 'Kaye', 'Underwood', '555-720-3848', 'home', datetime.datetime(2023, 1, 1, 0, 0), 235000]
[97143, 'Josiah', 'Vazquez', '211-391-1757', 'auto', datetime.datetime(2023, 1, 5, 0, 0), 17250]
[54621, 'Charles', 'Wise', '502-236-0425', 'home', datetime.datetime(2023, 1, 4, 0, 0), 1592000]


Quick query over the database to show all the rows, sorted by last name.

In [11]:
from sqlalchemy import select

stmt = select(policies).order_by(policies.c.last_name)
print(f'{stmt}\n')

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

SELECT policies.policy_id, policies.first_name, policies.last_name, policies.phone, policies.policy_type, policies.policy_date, policies.policy_value 
FROM policies ORDER BY policies.last_name

(17565, 'Charmaine', 'Hopkins', '206-566-6359', 'home', datetime.date(2023, 1, 2), 135000)
(10157, 'Jewel', 'Ingram', '598-338-6133', 'home', datetime.date(2023, 1, 6), 750000)
(21947, 'Bertram', 'Mcgee', '798-641-5925', 'home', datetime.date(2023, 1, 3), 550000)
(48918, 'Ernest', 'Mcneil', '349-711-8757', 'home', datetime.date(2023, 1, 1), 250000)
(66958, 'Brian', 'Patel', '368-889-1742', 'auto', datetime.date(2023, 1, 2), 32000)
(17108, 'Margarito', 'Rollins', '348-321-5711', 'auto', datetime.date(2023, 1, 4), 75000)
(98362, 'Miriam', 'Sutton', '361-863-4332', 'auto', datetime.date(2023, 1, 8), 21000)
(33372, 'Kaye', 'Underwood', '555-720-3848', 'home', datetime.date(2023, 1, 1), 235000)
(97143, 'Josiah', 'Vazquez', '211-391-1757', 'auto', datetime.date(2023, 1, 5), 17250)
(54621, 'Charles', '

## Create a mock API using Amazon API Gateway

Next, we want our LLM to be able to carry out actions using an API that we will provide.  

Using the Insurance Bot example, let's create a mock REST API using Amazon API Gateway.  

This API will have PUT and DELETE methods, to accept requests to modify or cancel an insurance policy. The API takes  an input request and returns a successful response.  Our API isn't backed by any functional logic for simplicity in the workshop environment.  However, a real implementation would be backed by a fully functional API. 

In [12]:
import boto3

apigateway = boto3.client('apigateway')

First, create the rest API

In [13]:
response = apigateway.create_rest_api(
    name='InsurancePolicyProcessing',
    description='API to process insurance policy requests',
    version='1.0',
    endpointConfiguration={
        'types': [
            'REGIONAL',
        ]
    }
)
print(response)

{'ResponseMetadata': {'RequestId': '092cf6fb-9b6b-4735-8662-8b3fb8b679df', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 02:56:55 GMT', 'content-type': 'application/json', 'content-length': '301', 'connection': 'keep-alive', 'x-amzn-requestid': '092cf6fb-9b6b-4735-8662-8b3fb8b679df', 'x-amz-apigw-id': 'NsqyqJuLPHcEbzg='}, 'RetryAttempts': 0}, 'id': 'pqraih5vog', 'name': 'InsurancePolicyProcessing', 'description': 'API to process insurance policy requests', 'createdDate': datetime.datetime(2023, 11, 1, 2, 56, 55, tzinfo=tzlocal()), 'version': '1.0', 'apiKeySource': 'HEADER', 'endpointConfiguration': {'types': ['REGIONAL']}, 'disableExecuteApiEndpoint': False, 'rootResourceId': '0cu1gxefef'}


In [14]:
rest_api_id = response['id']
print(rest_api_id)

# Get the rest api's root id
root_resource_id = apigateway.get_resources(
    restApiId=rest_api_id
)['items'][0]['id']

print(root_resource_id)

pqraih5vog
0cu1gxefef


Next, create the resources and methods associated with the previously created REST API.  

In [15]:
response = apigateway.create_resource(
    restApiId=rest_api_id,
    parentId=root_resource_id,
    pathPart='policy'
)

print(response)

{'ResponseMetadata': {'RequestId': 'db2d4fd8-2189-4193-a51b-bb37711b7ced', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 02:56:57 GMT', 'content-type': 'application/json', 'content-length': '77', 'connection': 'keep-alive', 'x-amzn-requestid': 'db2d4fd8-2189-4193-a51b-bb37711b7ced', 'x-amz-apigw-id': 'NsqzAJVPvHcEd5A='}, 'RetryAttempts': 0}, 'id': 'dyx1gi', 'parentId': '0cu1gxefef', 'pathPart': 'policy', 'path': '/policy'}


In [16]:
api_resource_id = response['id']
print(api_resource_id)

dyx1gi


In [17]:
# Add a put method to the rest api resource
api_put_method = apigateway.put_method(
  restApiId=rest_api_id,
  resourceId=api_resource_id,
  httpMethod='PUT',
  authorizationType='NONE',
  requestParameters={
      'method.request.querystring.policy': False
  }
)

api_put_response = apigateway.put_method_response(
    restApiId=rest_api_id,
    resourceId=api_resource_id,
    httpMethod='PUT',
    statusCode='200',
    responseModels={
        'application/json': 'Empty'
    }
)

print(api_put_method)
print(api_put_response)

{'ResponseMetadata': {'RequestId': '6fba9501-84a6-43a9-8b5b-ed806537cdff', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 02:56:59 GMT', 'content-type': 'application/json', 'content-length': '135', 'connection': 'keep-alive', 'x-amzn-requestid': '6fba9501-84a6-43a9-8b5b-ed806537cdff', 'x-amz-apigw-id': 'NsqzVL6SvHcEcgg='}, 'RetryAttempts': 0}, 'httpMethod': 'PUT', 'authorizationType': 'NONE', 'apiKeyRequired': False, 'requestParameters': {'method.request.querystring.policy': False}}
{'ResponseMetadata': {'RequestId': '073b451f-5683-41db-86dd-66dedf9269c8', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 02:56:59 GMT', 'content-type': 'application/json', 'content-length': '67', 'connection': 'keep-alive', 'x-amzn-requestid': '073b451f-5683-41db-86dd-66dedf9269c8', 'x-amz-apigw-id': 'NsqzWIl7vHcEdnQ='}, 'RetryAttempts': 0}, 'statusCode': '200', 'responseModels': {'application/json': 'Empty'}}


In [18]:
put_integration = apigateway.put_integration(
    restApiId=rest_api_id,
    resourceId=api_resource_id,
    httpMethod='PUT',
    type='MOCK',
    integrationHttpMethod='PUT',
    requestTemplates={
      "application/json": "{\"statusCode\": 200 }"
    }
)

put_integration_response = apigateway.put_integration_response(
    restApiId=rest_api_id,
    resourceId=api_resource_id,
    httpMethod='PUT',
    statusCode='200',
    responseTemplates={
        "application/json": "{\"statusCode\": 200 }"
    }
)

print(put_integration)
print(put_integration_response)

{'ResponseMetadata': {'RequestId': 'a01a7879-4c77-42ed-9b9b-67c26de10ed7', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 02:57:00 GMT', 'content-type': 'application/json', 'content-length': '193', 'connection': 'keep-alive', 'x-amzn-requestid': 'a01a7879-4c77-42ed-9b9b-67c26de10ed7', 'x-amz-apigw-id': 'NsqzgKn7vHcEdkg='}, 'RetryAttempts': 0}, 'type': 'MOCK', 'requestTemplates': {'application/json': '{"statusCode": 200 }'}, 'passthroughBehavior': 'WHEN_NO_MATCH', 'timeoutInMillis': 29000, 'cacheNamespace': 'dyx1gi', 'cacheKeyParameters': []}
{'ResponseMetadata': {'RequestId': '88b57636-5c6c-463a-8dc9-57263b7e0f2b', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 02:57:00 GMT', 'content-type': 'application/json', 'content-length': '87', 'connection': 'keep-alive', 'x-amzn-requestid': '88b57636-5c6c-463a-8dc9-57263b7e0f2b', 'x-amz-apigw-id': 'NsqzgJFyPHcEcOg='}, 'RetryAttempts': 0}, 'statusCode': '200', 'responseTemplates': {'application/json': '{"statu

In [19]:
# Add a delete method to the rest api resource
api_delete_method = apigateway.put_method(
  restApiId=rest_api_id,
  resourceId=api_resource_id,
  httpMethod='DELETE',
  authorizationType='NONE',
  requestParameters={
      'method.request.querystring.policy': False
  }
)

api_delete_response = apigateway.put_method_response(
    restApiId=rest_api_id,
    resourceId=api_resource_id,
    httpMethod='DELETE',
    statusCode='200',
    #responseParameters={
    #    'string': True|False
    #},
    responseModels={
        'application/json': 'Empty'
    }
)

print(api_delete_method)
print(api_delete_response)

{'ResponseMetadata': {'RequestId': 'bcdbd167-ccde-44b7-8155-f962acb88c81', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 02:57:03 GMT', 'content-type': 'application/json', 'content-length': '138', 'connection': 'keep-alive', 'x-amzn-requestid': 'bcdbd167-ccde-44b7-8155-f962acb88c81', 'x-amz-apigw-id': 'Nsqz5KTWPHcEddA='}, 'RetryAttempts': 0}, 'httpMethod': 'DELETE', 'authorizationType': 'NONE', 'apiKeyRequired': False, 'requestParameters': {'method.request.querystring.policy': False}}
{'ResponseMetadata': {'RequestId': '85ab69c2-ea2d-4ba4-805f-bf8be27126a1', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 02:57:03 GMT', 'content-type': 'application/json', 'content-length': '67', 'connection': 'keep-alive', 'x-amzn-requestid': '85ab69c2-ea2d-4ba4-805f-bf8be27126a1', 'x-amz-apigw-id': 'Nsqz6KiWPHcEb6w='}, 'RetryAttempts': 0}, 'statusCode': '200', 'responseModels': {'application/json': 'Empty'}}


In [20]:
delete_integration = apigateway.put_integration(
    restApiId=rest_api_id,
    resourceId=api_resource_id,
    httpMethod='DELETE',
    type='MOCK',
    integrationHttpMethod='DELETE',
    requestTemplates={
      "application/json": "{\"statusCode\": 200 }"      
    },
  )

delete_integration_response = apigateway.put_integration_response(
    restApiId=rest_api_id,
    resourceId=api_resource_id,
    httpMethod='DELETE',
    statusCode='200',
    responseTemplates={
        "application/json": "{\"statusCode\": 200 }"
    }
)

print(delete_integration)
print(delete_integration_response)

{'ResponseMetadata': {'RequestId': '1ba42a8d-ddc3-4b1f-949c-e7ddedea620a', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 02:57:04 GMT', 'content-type': 'application/json', 'content-length': '193', 'connection': 'keep-alive', 'x-amzn-requestid': '1ba42a8d-ddc3-4b1f-949c-e7ddedea620a', 'x-amz-apigw-id': 'Nsq0FJ2IvHcEdYQ='}, 'RetryAttempts': 0}, 'type': 'MOCK', 'requestTemplates': {'application/json': '{"statusCode": 200 }'}, 'passthroughBehavior': 'WHEN_NO_MATCH', 'timeoutInMillis': 29000, 'cacheNamespace': 'dyx1gi', 'cacheKeyParameters': []}
{'ResponseMetadata': {'RequestId': 'f92bae72-2104-414c-861c-88a2a6af7d29', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 02:57:04 GMT', 'content-type': 'application/json', 'content-length': '87', 'connection': 'keep-alive', 'x-amzn-requestid': 'f92bae72-2104-414c-861c-88a2a6af7d29', 'x-amz-apigw-id': 'Nsq0GJqAPHcEcWg='}, 'RetryAttempts': 0}, 'statusCode': '200', 'responseTemplates': {'application/json': '{"statu

Finally, deploy the API to create an endpoint.  This endpoint will be used later in the notebook when we configure the tools that will be made available to our LLM

In [21]:
# Deploy API

deploy_api = apigateway.create_deployment(
    restApiId=rest_api_id,
    stageName='Mock-Test',
    stageDescription='Mock Testing for Insurance API',
    description='Mock Testing API',
)

In [22]:
print(deploy_api)

{'ResponseMetadata': {'RequestId': 'ab72a56f-8fcc-43db-9773-fa9e42bcfd77', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 01 Nov 2023 02:57:05 GMT', 'content-type': 'application/json', 'content-length': '74', 'connection': 'keep-alive', 'x-amzn-requestid': 'ab72a56f-8fcc-43db-9773-fa9e42bcfd77', 'x-amz-apigw-id': 'Nsq0RKTjPHcEdgQ='}, 'RetryAttempts': 0}, 'id': 'z9sj10', 'description': 'Mock Testing API', 'createdDate': datetime.datetime(2023, 11, 1, 2, 57, 5, tzinfo=tzlocal())}


In [23]:
deployment_id = deploy_api['id']
print(deployment_id)

z9sj10


In [24]:
# Get URLs
region = boto3.Session().region_name

request_url = f"https://{rest_api_id}.execute-api.{region}.amazonaws.com/Mock-Test/policy"
print(request_url)

https://pqraih5vog.execute-api.us-west-2.amazonaws.com/Mock-Test/policy


# Prepare the Models

## LLaMa-2-13B-Chat

To show diversity in approaches, you will use a local LLaMa-2-13B-Chat model to generate SQL to query the database you just made. This will be a 5-bit quantized version of the model, running on your notebook CPU.

This GGUF version of the model can be found at [TheBloke/Llama-2-13B-chat-GGUF](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF) in the HuggingFace Model Repo.

This model is a quantized version of [Meta's LLaMa-2-13B-Chat](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf).

In [25]:
from huggingface_hub import hf_hub_download
from pathlib import Path

local_model_dir = "models"
model_repo = "TheBloke/Llama-2-13b-Chat-GGUF"
model_filename = "llama-2-13b-chat.Q5_K_M.gguf"

model_filepath = Path(local_model_dir + "/" + model_filename)

hf_hub_download(repo_id=model_repo, filename=model_filename, local_dir=local_model_dir)

print(model_filepath)

models/llama-2-13b-chat.Q5_K_M.gguf


In [26]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
# Callbacks support token-wise streaming if desired
callback_manager = CallbackManager([])

# Make sure the model path is correct for your system!
llama_llm = LlamaCpp(
    model_path=str(model_filepath),
    temperature=0.001,
    max_tokens=256,
    top_p=1,
    callback_manager=callback_manager, 
    verbose=True, # Verbose is required to pass to the callback manager
    n_ctx=4096,
)


llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from models/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  5120,  5120,     1,

## Anthropic's Claude V2 via Amazon Bedrock

In this example you will use Anthropic's Claude V2 as the LLM for the reasoning part of the ReAct approach in the form of a [Langchain Agent](https://python.langchain.com/docs/modules/agents/). This model will be responsible for accepting the request, breaking down the chain of thought reasoning, selecting tools, and formulate a final response.

[Amazon Bedrock](https://aws.amazon.com/bedrock/) will provide the interface for Claude in this example, showing how you can leverage this model (and others) though an API instead of hosting your own endpoints.

In [27]:
from langchain.llms import Bedrock

llm = Bedrock(
    model_id="anthropic.claude-v2", 
    model_kwargs={'max_tokens_to_sample':10000}
)

## Interfacing with a database via Langchain SQLDatabaseChain

In this section you will build the foudational element for a tool you will create later, which will take a natural language query and return a response from the in-memory SQLite database.

You'll accomplish this by creating a Langchain SQLDatabase chain.

In [28]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

Below you can see an example prompt for your SQLDatabaseChain.

It accepts:
- `dialect`: parameter for the dialect of the target database
- `table_info`: parameter that will autogenerate a sample of the schemas and first few rows of data from the tables
- `input`: parameter for the input NLP query to generate SQL from.

In [29]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain

_DEFAULT_TEMPLATE = """
You are a {dialect} expert. Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per {dialect}. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (\") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves \"today\".

Once you have a SQLResult, use it to formulate a natural language answer. Example: The policy id is 12345.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
{table_info}

Question: {input}"""

PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

Create a SQLDatabase object from the DB engine created earlier for the SQLite database.

In [30]:
db = SQLDatabase(engine)

You can test your SQLDatabaseChain with a sample NLP query that will be the foundation of a later example:

In [31]:
sql_chain = SQLDatabaseChain.from_llm(llm=llama_llm, db=db, verbose=True, prompt=PROMPT)

You can test your SQLDatabaseChain with a sample NLP query that will be the foundation of a later example:

In [32]:
sql_chain.run(query="What is Charles Wise's policy id?")

[chain/start] [1:chain:SQLDatabaseChain] Entering Chain run with input:
{
  "query": "What is Charles Wise's policy id?"
}
[chain/start] [1:chain:SQLDatabaseChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is Charles Wise's policy id?\nSQLQuery:",
  "top_k": "5",
  "dialect": "sqlite",
  "table_info": "\nCREATE TABLE policies (\n\tpolicy_id INTEGER NOT NULL, \n\tfirst_name VARCHAR(50) NOT NULL, \n\tlast_name VARCHAR(50) NOT NULL, \n\tphone VARCHAR(15) NOT NULL, \n\tpolicy_type VARCHAR(25) NOT NULL, \n\tpolicy_date DATE NOT NULL, \n\tpolicy_value INTEGER NOT NULL, \n\tPRIMARY KEY (policy_id)\n)\n\n/*\n3 rows from policies table:\npolicy_id\tfirst_name\tlast_name\tphone\tpolicy_type\tpolicy_date\tpolicy_value\n10157\tJewel\tIngram\t598-338-6133\thome\t2023-01-06\t750000\n17108\tMargarito\tRollins\t348-321-5711\tauto\t2023-01-04\t75000\n17565\tCharmaine\tHopkins\t206-566-6359\thome\t2023-01-02\t135000\n*/",
  "stop": [
    "\nSQLResult:"
  ]
}
[llm/start] [1:cha


llama_print_timings:        load time =   808.58 ms
llama_print_timings:      sample time =    17.14 ms /    19 runs   (    0.90 ms per token,  1108.19 tokens per second)
llama_print_timings: prompt eval time = 43218.41 ms /   580 tokens (   74.51 ms per token,    13.42 tokens per second)
llama_print_timings:        eval time =  2077.20 ms /    18 runs   (  115.40 ms per token,     8.67 tokens per second)
llama_print_timings:       total time = 45507.57 ms
Llama.generate: prefix-match hit


[llm/end] [1:chain:SQLDatabaseChain > 2:chain:LLMChain > 3:llm:LlamaCpp] [45.51s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " SELECT policy_id FROM policies WHERE first_name = 'Charles';",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:SQLDatabaseChain > 2:chain:LLMChain] [45.51s] Exiting Chain run with output:
{
  "text": " SELECT policy_id FROM policies WHERE first_name = 'Charles';"
}
[chain/start] [1:chain:SQLDatabaseChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is Charles Wise's policy id?\nSQLQuery:SELECT policy_id FROM policies WHERE first_name = 'Charles';\nSQLResult: [(54621,)]\nAnswer:",
  "top_k": "5",
  "dialect": "sqlite",
  "table_info": "\nCREATE TABLE policies (\n\tpolicy_id INTEGER NOT NULL, \n\tfirst_name VARCHAR(50) NOT NULL, \n\tlast_name VARCHAR(50) NOT NULL, \n\tphone VARCHAR(15) NOT NULL, \n\tpolicy_type 


llama_print_timings:        load time =   808.58 ms
llama_print_timings:      sample time =    10.88 ms /    12 runs   (    0.91 ms per token,  1102.43 tokens per second)
llama_print_timings: prompt eval time =  2161.87 ms /    30 tokens (   72.06 ms per token,    13.88 tokens per second)
llama_print_timings:        eval time =  1353.57 ms /    11 runs   (  123.05 ms per token,     8.13 tokens per second)
llama_print_timings:       total time =  3561.66 ms


'The policy id is 54621.'

After running the chain, you can see that it returns a policy id for the person in question. Sometimes you may see that the resulting query is not specfic enough (maybe only looking at `first_name` instead of both `first_name` and `last_name`), but this can be addressed by refining the prompt.

However, it will suffice for this example.

## Build a Langchain APIChain

Similar to the previous example showing the use of the SQLLite database in a chain, we'll show how you can utilize the previously created API tool in the context of a chain as well.  This section is showing an APIChain for demonstration purposes but later you'll see how we can use the API as a defined tool as part of our ReAct workflow.  

In this example, we'll read in API documentation using an explicit copy of the API's documentation (explicit_docs.txt).  Alternatively, you could have the LLM read this directly.  

In [33]:
api_docs = open('explicit_docs.txt', 'r')
info = api_docs.read()
explicit_docs=info.replace('\n', '')
api_docs.close()

In [34]:
from langchain.chains import APIChain

API_chain = APIChain.from_llm_and_api_docs(llm, explicit_docs, verbose=True)

In [35]:
response = API_chain.run("Request policy cancellation for policy id 54621")
print(response)

[chain/start] [1:chain:APIChain] Entering Chain run with input:
{
  "question": "Request policy cancellation for policy id 54621"
}
[chain/start] [1:chain:APIChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "question": "Request policy cancellation for policy id 54621",
  "api_docs": "description: Insurance Policy API to update,cancel policyUpdate Method: PUT Cancel Method: DELETE Get Method: GET Sample Base API Endpoint: https://qglixpuz4c.execute-api.us-west-2.amazonaws.com/Mock-Test/policy? Sample Update Endpoint: https://qglixpuz4c.execute-api.us-west-2.amazonaws.com/Mock-Test/policy?PUT/4444/7203421165Sample Delete Endpoint: https://qglixpuz4c.execute-api.us-west-2.amazonaws.com/Mock-Test/policy?DELETE/4444Response Format: The response format for this API is in JSON.Authentication: No authentication is required to use this API.Error Codes: The following response codes may be returned by the API:• 200 OK: Successful request.• 400 Bad Request: Invalid parameters.• 401 Una

## Setting up Tools

With the foundational components now in place, you will begin setting up the concept of tools. Tools are purpose built components to satisfy a particular need. They can look up data, run API commands, or even go perform searches on the internet to satisfy the needs of the reasoning chain of thought.

As you proceed you'll see how to built an extensible toolbox that will attempt to automatically select the right tools based on the input query.

### Create API Tool

First, we'll define the API Tool that will be made available to our ReAct workflow.   This API tool will utilize the previously created insurance policy API.  

For this, we are using LangChain's [StructuredTool](https://blog.langchain.dev/structured-tools/) which allows us to represent a function as a tool that an agent can easily interface with to perform actions.

In [36]:
request_url

'https://pqraih5vog.execute-api.us-west-2.amazonaws.com/Mock-Test/policy'

In [37]:
import requests
from langchain.tools import StructuredTool

# A structured tool represents an action an agent can take. It wraps any function you provide to let an agent easily interface with it.
    
def cancel_policy_request(request: str): 
    """Sends a DELETE request to the policy API with the provided policy id"""
    url = request_url
    policy_id = request.rsplit(None, 1)[-1]
    result = requests.delete(url+"?DELETE/"+policy_id)
    return f"Successfully submitted policy cancellation for: {policy_id}, Status: {result.status_code} - {result.text}"

def update_policy_request(request: str):
    """Sends a PUT request to the policy API with the provided policy id and data to update"""
    url = request_url
    policy_id = request[request.find("policy")+len("policy"):].split()[0]
    update_field = request[request.find("update the")+len("update the"):].split()[0]
    update_data = request.rsplit(None, 1)[-1]
    result = requests.put(url+"?PUT/"+policy_id, update_field+"/"+update_data)
    return f"Successfully submitted update for: {policy_id}, Update for: {update_field}, {update_data}, Status: {result.status_code} - {result.text}"

In [38]:
update_policy_request = StructuredTool.from_function(update_policy_request)
cancel_policy_request = StructuredTool.from_function(cancel_policy_request)

Test the newly created tool

In [39]:
cancel_policy_request.run('the policy id is 54621')

[tool/start] [1:tool:cancel_policy_request] Entering Tool run with input:
"the policy id is 54621"
[tool/end] [1:tool:cancel_policy_request] s] Exiting Tool run with output:
"Successfully submitted policy cancellation for: 54621, Status: 200 - {"statusCode": 200 }"


'Successfully submitted policy cancellation for: 54621, Status: 200 - {"statusCode": 200 }'

In [40]:
update_policy_request.run('Submit a request to update the phone number in policy 54621 to 333-321-5622')

[tool/start] [1:tool:update_policy_request] Entering Tool run with input:
"Submit a request to update the phone number in policy 54621 to 333-321-5622"
[tool/end] [1:tool:update_policy_request] s] Exiting Tool run with output:
"Successfully submitted update for: 54621, Update for: phone, 333-321-5622, Status: 200 - {"statusCode": 200 }"


'Successfully submitted update for: 54621, Update for: phone, 333-321-5622, Status: 200 - {"statusCode": 200 }'

Let's now take all of the tools that we've created and add them all to the list of tools that will be made available to the LLM.

In [41]:
from langchain.chains import LLMMathChain
from langchain.agents import load_tools
from langchain.tools import Tool

llm_math = LLMMathChain.from_llm(llm, verbose=True)

ALL_TOOLS = [
        Tool(
            name="calculator",
            func=llm_math.run,
            description="Useful when you need to perform mathematical operations."
        ),    
        Tool(
            name="insurance_policy_lookup",
            func=sql_chain.run,
            description="Useful when you need to look up insurance policy information. This tool takes in a full question about customers and their policies and will return a policy id. Example: [What is the policy id for Jane Doe?, What is Cynthia Stone's policy?]"
        ),
        Tool(
            name="cancel_policy_request",
            func=cancel_policy_request.run,
            description="Useful when you need to submit a request to cancel an insurance policy. This tool takes in the customer's policy id and will return an API response message. Example: [Submit a request to cancel policy for policy id of 4321]"
        ),
        Tool(
            name="update_policy_request",
            func=update_policy_request.run,
            description="Useful when you need to submit a request to update an insurance policy. This tool takes in the customer's policy id to be updated as well as data to be updated. An API response will be returned.  Example: [Submit a request to update the phone number in policy 4321 to 455-255-5555]"
        ),
]

## Dynamic Tool Selection

An [embedding model](https://huggingface.co/blog/getting-started-with-embeddings) takes a text string and converts it into a numerical vector representation, which will allow you to store it in a vector database and query it based on semantic similarity.

Here, you will host a local [MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) embedding model, which will generate a set of embeddings to help semantically search for the right tools in the example.

**Note: Ignore any "Error displaying widget: model not found" errors you may see here.**

In [42]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

The vector store method of tool selection in this example uses [Meta's Facebook AI Similarity Search (FAISS)](https://github.com/facebookresearch/faiss) as a vector database. There are many other options available as well.

First you will create a set of documents (one for each tool), that you will vectorize and use to determine best match for tools.

In [43]:
from langchain.vectorstores import FAISS
from langchain.schema import Document

docs = [
    Document(page_content=t.description, metadata={"index": i})
    for i, t in enumerate(ALL_TOOLS)
]

docs

[Document(page_content='Useful when you need to perform mathematical operations.', metadata={'index': 0}),
 Document(page_content="Useful when you need to look up insurance policy information. This tool takes in a full question about customers and their policies and will return a policy id. Example: [What is the policy id for Jane Doe?, What is Cynthia Stone's policy?]", metadata={'index': 1}),
 Document(page_content="Useful when you need to submit a request to cancel an insurance policy. This tool takes in the customer's policy id and will return an API response message. Example: [Submit a request to cancel policy for policy id of 4321]", metadata={'index': 2}),
 Document(page_content="Useful when you need to submit a request to update an insurance policy. This tool takes in the customer's policy id to be updated as well as data to be updated. An API response will be returned.  Example: [Submit a request to update the phone number in policy 4321 to 455-255-5555]", metadata={'index': 3

FAISS provides a `from_documents` method for automatically generating embeddings and storing them in the index.

In [44]:
vector_store = FAISS.from_documents(docs, embeddings)

In this section you will define the `get_tools` method, which will embed the input query and find relevant documents that are semantically close. This will allow you to take large lists of tools and scope them down to the most relevant ones.

You'll use this method to demonstrate tool scoping below and it will also be used in supplying a list of tools to your agent later.

In [45]:
retriever = vector_store.as_retriever()


def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [ALL_TOOLS[d.metadata["index"]] for d in docs]

def print_tools(tools_arr):
    for tool in tools_arr:
        print(f'{tool}\n\n')

Let's see if we can find the right tool for a given input...

Here you get a list of tools for a mathematical input query. Notice that the `calculator` tool is listed first as its description most closely matches the request. 

You can also control how many results to return by modifying the `get_relevant_documents` request inside of `get_tools`.

In [46]:
print_tools(get_tools("What is the policy id for Bob Jones?"))

name='insurance_policy_lookup' description="Useful when you need to look up insurance policy information. This tool takes in a full question about customers and their policies and will return a policy id. Example: [What is the policy id for Jane Doe?, What is Cynthia Stone's policy?]" func=<bound method Chain.run of SQLDatabaseChain(verbose=True, llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['dialect', 'input', 'table_info'], template='\nYou are a {dialect} expert. Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per {dialect}. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wr

Alternatively, the `insurance_policy_lookup` tool is a better match for a policy related inquiry.

In [47]:
print_tools(get_tools("Cancel the policy for policy id 4325"))

name='cancel_policy_request' description="Useful when you need to submit a request to cancel an insurance policy. This tool takes in the customer's policy id and will return an API response message. Example: [Submit a request to cancel policy for policy id of 4321]" func=<bound method BaseTool.run of StructuredTool(name='cancel_policy_request', description='cancel_policy_request(request: str) - Sends a DELETE request to the policy API with the provided policy id', args_schema=<class 'pydantic.v1.main.cancel_policy_requestSchemaSchema'>, func=<function cancel_policy_request at 0x7fc252fc39a0>)>


name='update_policy_request' description="Useful when you need to submit a request to update an insurance policy. This tool takes in the customer's policy id to be updated as well as data to be updated. An API response will be returned.  Example: [Submit a request to update the phone number in policy 4321 to 455-255-5555]" func=<bound method BaseTool.run of StructuredTool(name='update_policy_re

# Agent Setup

With you models ready to go, and your toolbox configured, you're ready to setup your ReAct agent.

The agent will take in a ReAct style prompt along with the list of tools to build out the chain of thought reasoning and resulting actions.

The Langchain agent initialization takes 3 prompt keyword arguments to customize the agent's prompt:
- `prefix`: the beginning of the prompt
- \<list of tools and their descriptions is automatically injected here\>
- `format_instructions`: how to format the intermediary and final responses, this is important in dictating how the chain of thought is created and processed.
- `suffix`: The invocation of the agent. This contains the initial input request as well as the conversational chain to determine what the agent has already done and seen.

Some template parameters you see below are:
- `{tool_names}`: this is a list of the tools that are available to the agent, in name only. This helps direct the agent into what to supply as actions during reasoning. In this case the agent will do this step for you, but in others you may need to extract the names yourself.
- `{agent_scratchpad}`: this is important as it documents the activities and obserations that the agent has had throughout its reasoning. Without this, you will normally see the agent endlessly loop.
- `{input}`: The input request from the user.

In [48]:
prefix = """Human: You are an agent tasked with helping look up and modify insurance claims.

Given an input request, take a step-by-step approach to find an insurance policy and modify its status.
Only use the tools provided.
When you have completed the task, end your chain of thought and provide a final response to the user.

You have access to the following tools:"""

format_instructions="""

Use the following format:


To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```
"""

suffix="""
Begin!

Previous conversation history:{agent_scratchpad}
Original input: {input}

Assistant:
"""

In [49]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

This is the user request that the agent will process using the provided tools.

In [50]:
query = "Cancel Charles Wise's insurance policy"

Here you will set up your agent.

initialize_agent has the following configuration:
- `agent`: using the [ZERO_SHOT_REACT agent](https://python.langchain.com/docs/modules/agents/agent_types/react#using-zeroshotreactagent)
- `agent_kwargs`: variable contains the object with all of the customized prompt information.
- `tools`: list of tools from the vector store, dynamically generated from the input query
- `llm`: the agent LLM (in this case Claude V2 from Amazon Bedrock)
- `verbose`: show all the details for learning purposes
- `return_intermediate_steps`: important for retaining all of the thoughts/actions/observations in the agent_scratchpad
- `max_iterations`: VERY IMPORTANT for ensuring that your agent doesn't endlessly loop and run away. Prevents the agent from hammering on the associated LLM.
- `handle_parsing_errors`: used for dealing with issues parsing the output of a given step, not necessary but catches edge cases in this example.

In [51]:
tools = get_tools(query)

agent_kwargs = {
    'prefix':prefix,
    'suffix':suffix,
    'format_instructions':format_instructions
}

zero_shot_agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs = agent_kwargs,
    tools=tools,
    llm=llm,
    verbose=True,
    return_intermediate_steps=True,
    max_iterations=5,
    handle_parsing_errors=True
)

Finally you can invoke your agent with the input query!

The output will be verbose, but if you follow through you will see the chain start/end and invoke tools to get the information and take action until it reaches a conclusion or hits the `max_iterations`.

In [52]:
zero_shot_agent(query)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Cancel Charles Wise's insurance policy"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Cancel Charles Wise's insurance policy",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:Bedrock] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent tasked with helping look up and modify insurance claims.\n\nGiven an input request, take a step-by-step approach to find an insurance policy and modify its status.\nOnly use the tools provided.\nWhen you have completed the task, end your chain of thought and provide a final response to the user.\n\nYou have access to the following tools:\n\ncancel_policy_request: Useful when you need to submit a request to cancel an insurance policy. This tool takes in the customer's policy id and will return

Llama.generate: prefix-match hit

llama_print_timings:        load time =   808.58 ms
llama_print_timings:      sample time =    17.33 ms /    19 runs   (    0.91 ms per token,  1096.11 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  2323.28 ms /    19 runs   (  122.28 ms per token,     8.18 tokens per second)
llama_print_timings:       total time =  2387.85 ms
Llama.generate: prefix-match hit


[llm/end] [1:chain:AgentExecutor > 4:tool:insurance_policy_lookup > 5:chain:SQLDatabaseChain > 6:chain:LLMChain > 7:llm:LlamaCpp] [2.39s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " SELECT policy_id FROM policies WHERE first_name = 'Charles';",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 4:tool:insurance_policy_lookup > 5:chain:SQLDatabaseChain > 6:chain:LLMChain] [2.39s] Exiting Chain run with output:
{
  "text": " SELECT policy_id FROM policies WHERE first_name = 'Charles';"
}
[chain/start] [1:chain:AgentExecutor > 4:tool:insurance_policy_lookup > 5:chain:SQLDatabaseChain > 8:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is Charles Wise's policy id?\nSQLQuery:SELECT policy_id FROM policies WHERE first_name = 'Charles';\nSQLResult: [(54621,)]\nAnswer:",
  "top_k": "5",
  "dialect": "sqlite",
  "table_info": "\nCREATE 


llama_print_timings:        load time =   808.58 ms
llama_print_timings:      sample time =    10.89 ms /    12 runs   (    0.91 ms per token,  1102.23 tokens per second)
llama_print_timings: prompt eval time =  2169.69 ms /    30 tokens (   72.32 ms per token,    13.83 tokens per second)
llama_print_timings:        eval time =  1312.47 ms /    11 runs   (  119.32 ms per token,     8.38 tokens per second)
llama_print_timings:       total time =  3530.14 ms


[llm/end] [1:chain:AgentExecutor > 4:tool:insurance_policy_lookup > 5:chain:SQLDatabaseChain > 8:chain:LLMChain > 9:llm:LlamaCpp] [3.53s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " The policy id is 54621.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 4:tool:insurance_policy_lookup > 5:chain:SQLDatabaseChain > 8:chain:LLMChain] [3.53s] Exiting Chain run with output:
{
  "text": " The policy id is 54621."
}
[chain/end] [1:chain:AgentExecutor > 4:tool:insurance_policy_lookup > 5:chain:SQLDatabaseChain] [5.93s] Exiting Chain run with output:
{
  "result": "The policy id is 54621."
}
[tool/end] [1:chain:AgentExecutor > 4:tool:insurance_policy_lookup] [5.93s] Exiting Tool run with output:
"The policy id is 54621."
[chain/start] [1:chain:AgentExecutor > 10:chain:LLMChain] Entering Chain run with input:
{
  "input": "Cancel Charles Wise's insura

Llama.generate: prefix-match hit

llama_print_timings:        load time =   808.58 ms
llama_print_timings:      sample time =    17.42 ms /    19 runs   (    0.92 ms per token,  1090.45 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  2193.88 ms /    19 runs   (  115.47 ms per token,     8.66 tokens per second)
llama_print_timings:       total time =  2259.20 ms
Llama.generate: prefix-match hit


[llm/end] [1:chain:AgentExecutor > 16:tool:insurance_policy_lookup > 17:chain:SQLDatabaseChain > 18:chain:LLMChain > 19:llm:LlamaCpp] [2.26s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " SELECT policy_id FROM policies WHERE first_name = 'Charles';",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 16:tool:insurance_policy_lookup > 17:chain:SQLDatabaseChain > 18:chain:LLMChain] [2.26s] Exiting Chain run with output:
{
  "text": " SELECT policy_id FROM policies WHERE first_name = 'Charles';"
}
[chain/start] [1:chain:AgentExecutor > 16:tool:insurance_policy_lookup > 17:chain:SQLDatabaseChain > 20:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is Charles Wise's policy id?\nSQLQuery:SELECT policy_id FROM policies WHERE first_name = 'Charles';\nSQLResult: [(54621,)]\nAnswer:",
  "top_k": "5",
  "dialect": "sqlite",
  "table_info": 


llama_print_timings:        load time =   808.58 ms
llama_print_timings:      sample time =    10.71 ms /    12 runs   (    0.89 ms per token,  1120.34 tokens per second)
llama_print_timings: prompt eval time =  2428.50 ms /    30 tokens (   80.95 ms per token,    12.35 tokens per second)
llama_print_timings:        eval time =  1257.46 ms /    11 runs   (  114.31 ms per token,     8.75 tokens per second)
llama_print_timings:       total time =  3732.73 ms


[llm/end] [1:chain:AgentExecutor > 22:chain:LLMChain > 23:llm:Bedrock] [4.24s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Here is my continued step-by-step thought process:\n\nThought: Do I need to use a tool? Yes\nAction: cancel_policy_request\nAction Input: Submit a request to cancel policy for policy id of 54621  ",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 22:chain:LLMChain] [4.24s] Exiting Chain run with output:
{
  "text": " Here is my continued step-by-step thought process:\n\nThought: Do I need to use a tool? Yes\nAction: cancel_policy_request\nAction Input: Submit a request to cancel policy for policy id of 54621  "
}
[tool/start] [1:chain:AgentExecutor > 24:tool:cancel_policy_request] Entering Tool run with input:
"Submit a request to cancel policy for policy id of 54621"
[tool/start] [1:chain:AgentExecutor > 24:tool:cancel_p

{'input': "Cancel Charles Wise's insurance policy",
 'output': "Based on the previous conversation, I looked up Charles Wise's policy id using the insurance_policy_lookup tool, which returned id 54621. I then submitted a request to cancel policy 54621 using the cancel_policy_request tool, which returned a 200 success status. The original request was to cancel Charles Wise's insurance policy, which I have completed by looking up his policy id and submitting a cancellation request for that id.",
 'intermediate_steps': [(AgentAction(tool='insurance_policy_lookup', tool_input="What is Charles Wise's policy id?", log=" Thought: Do I need to use a tool? Yes\nAction: insurance_policy_lookup \nAction Input: What is Charles Wise's policy id?"),
   'The policy id is 54621.'),
  (AgentAction(tool='cancel_policy_request', tool_input='Submit a request to cancel policy for policy id of 54621', log=' Thought: Do I need to use a tool? Yes\nAction: cancel_policy_request\nAction Input: Submit a request 

Let's try to process an update for Charles...

In [53]:
query = "Update the insurance policy for Charles Wise by updating the phone number to 333-321-5622"

In [54]:
zero_shot_agent(query)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Update the insurance policy for Charles Wise by updating the phone number to 333-321-5622"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Update the insurance policy for Charles Wise by updating the phone number to 333-321-5622",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:Bedrock] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent tasked with helping look up and modify insurance claims.\n\nGiven an input request, take a step-by-step approach to find an insurance policy and modify its status.\nOnly use the tools provided.\nWhen you have completed the task, end your chain of thought and provide a final response to the user.\n\nYou have access to the following tools:\n\ncancel_policy_request: Useful when you need to submi

Llama.generate: prefix-match hit


[llm/end] [1:chain:AgentExecutor > 4:tool:insurance_policy_lookup > 5:chain:SQLDatabaseChain > 6:chain:LLMChain > 7:llm:LlamaCpp] [2.25s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " SELECT policy_id FROM policies WHERE first_name = 'Charles';",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 4:tool:insurance_policy_lookup > 5:chain:SQLDatabaseChain > 6:chain:LLMChain] [2.25s] Exiting Chain run with output:
{
  "text": " SELECT policy_id FROM policies WHERE first_name = 'Charles';"
}
[chain/start] [1:chain:AgentExecutor > 4:tool:insurance_policy_lookup > 5:chain:SQLDatabaseChain > 8:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is Charles Wise's policy id?\nSQLQuery:SELECT policy_id FROM policies WHERE first_name = 'Charles';\nSQLResult: [(54621,)]\nAnswer:",
  "top_k": "5",
  "dialect": "sqlite",
  "table_info": "\nCREATE 


llama_print_timings:        load time =   808.58 ms
llama_print_timings:      sample time =    17.07 ms /    19 runs   (    0.90 ms per token,  1113.00 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  2179.56 ms /    19 runs   (  114.71 ms per token,     8.72 tokens per second)
llama_print_timings:       total time =  2244.15 ms
Llama.generate: prefix-match hit


[llm/end] [1:chain:AgentExecutor > 4:tool:insurance_policy_lookup > 5:chain:SQLDatabaseChain > 8:chain:LLMChain > 9:llm:LlamaCpp] [3.54s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " The policy id is 54621.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 4:tool:insurance_policy_lookup > 5:chain:SQLDatabaseChain > 8:chain:LLMChain] [3.54s] Exiting Chain run with output:
{
  "text": " The policy id is 54621."
}
[chain/end] [1:chain:AgentExecutor > 4:tool:insurance_policy_lookup > 5:chain:SQLDatabaseChain] [5.79s] Exiting Chain run with output:
{
  "result": "The policy id is 54621."
}
[tool/end] [1:chain:AgentExecutor > 4:tool:insurance_policy_lookup] [5.79s] Exiting Tool run with output:
"The policy id is 54621."
[chain/start] [1:chain:AgentExecutor > 10:chain:LLMChain] Entering Chain run with input:
{
  "input": "Update the insurance policy 


llama_print_timings:        load time =   808.58 ms
llama_print_timings:      sample time =    10.59 ms /    12 runs   (    0.88 ms per token,  1133.36 tokens per second)
llama_print_timings: prompt eval time =  2142.23 ms /    30 tokens (   71.41 ms per token,    14.00 tokens per second)
llama_print_timings:        eval time =  1346.10 ms /    11 runs   (  122.37 ms per token,     8.17 tokens per second)
llama_print_timings:       total time =  3534.29 ms


[llm/end] [1:chain:AgentExecutor > 10:chain:LLMChain > 11:llm:Bedrock] [3.26s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Thought: Do I need to use a tool? Yes\nAction: update_policy_request\nAction Input: Submit a request to update the phone number in policy 54621 to 333-321-5622",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 10:chain:LLMChain] [3.26s] Exiting Chain run with output:
{
  "text": " Thought: Do I need to use a tool? Yes\nAction: update_policy_request\nAction Input: Submit a request to update the phone number in policy 54621 to 333-321-5622"
}
[tool/start] [1:chain:AgentExecutor > 12:tool:update_policy_request] Entering Tool run with input:
"Submit a request to update the phone number in policy 54621 to 333-321-5622"
[tool/start] [1:chain:AgentExecutor > 12:tool:update_policy_request > 13:tool:update_policy_request] Entering

{'input': 'Update the insurance policy for Charles Wise by updating the phone number to 333-321-5622',
 'output': "I looked up Charles Wise's policy id using the insurance policy lookup tool and got id 54621. I then submitted a request using the update policy request tool to update the phone number in policy 54621 to 333-321-5622. The request was successful with a 200 response code. I have completed the original request to update Charles Wise's insurance policy phone number to 333-321-5622.",
 'intermediate_steps': [(AgentAction(tool='insurance_policy_lookup', tool_input="What is Charles Wise's policy id?", log=" Thought: Do I need to use a tool? Yes\nAction: insurance_policy_lookup \nAction Input: What is Charles Wise's policy id?"),
   'The policy id is 54621.'),
  (AgentAction(tool='update_policy_request', tool_input='Submit a request to update the phone number in policy 54621 to 333-321-5622', log=' Thought: Do I need to use a tool? Yes\nAction: update_policy_request\nAction Input:

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools( agent=agent, tools=tools, verbose=True )

In [ ]:
agent_executor.run(query)